In [1]:
import numpy as np
from pprint import pprint

import craft_text_detector as craft
from craft_text_detector.word_to_line import sort_bbs_line_by_line

#### set image path and export folder directory

In [2]:
image_name = 'idcard.png'
image_path = 'figures/' + image_name
output_dir = 'outputs/'

#### read image

In [3]:
image = craft.read_image(image_path)

#### load models

In [4]:
craft_model_path = "../craft_mlt_25k.pth"
refinenet_model_path = "../craft_refiner_CTW1500.pth"
craft_net = craft.craft_detector(image=image,
                                 craft_model_path=craft_model_path,
                                 refinenet_model_path=refinenet_model_path,
                                 cuda=True)


#### perform prediction

In [5]:
text_threshold = 0.9
link_threshold = 0.2
low_text = 0.2
cuda = True  # False
show_time = False
# perform prediction
prediction_result = craft_net(image=image,
                         text_threshold=0.7,
                         link_threshold=0.4,
                         low_text=0.4,
                         square_size=1280,
                         show_time=True)


infer/postproc time : 0.144/0.077


### Inspect predicted results

In [6]:
keys = prediction_result.keys()
heatmap_keys = prediction_result["heatmaps"].keys()
pprint(keys)
pprint(heatmap_keys)

dict_keys(['boxes', 'boxes_as_ratios', 'polys', 'polys_as_ratios', 'heatmaps', 'times'])
dict_keys(['text_score_heatmap', 'link_score_heatmap'])


In [7]:
boxes = prediction_result['boxes']
boxes_as_ratios = prediction_result['boxes_as_ratios']
polys = prediction_result['polys']
polys_as_ratios = prediction_result['polys_as_ratios']
text_score_heatmap = prediction_result['heatmaps']['text_score_heatmap']
link_score_heatmap = prediction_result['heatmaps']['link_score_heatmap']
pprint(boxes.shape)
pprint(boxes_as_ratios.shape)
pprint(polys.shape)
pprint(polys_as_ratios.shape)
pprint(text_score_heatmap.shape)
pprint(link_score_heatmap.shape)

(19, 4, 2)
(19, 4, 2)
(19,)
(19,)
(256, 400, 3)
(256, 400, 3)


In [9]:
coords = np.array(boxes).astype(np.int32)
pprint(coords.shape)
# pprint(coords[0:5])

# line_bbs = sort_bbs_line_by_line(boxes)

def coord_sort(boxes):
    line=[]
    lines=[]
    number_of_box = boxes.shape[0]
    # all bounding boxes.
    for i, box in enumerate(boxes):
        previous_box = None
        box_x0, box_y0 = box[0::2]  # top- left corner <-> bottom- right corner
        if previous_box is not None:
            pass

coord_sort(boxes)

(19, 4, 2)


KeyboardInterrupt: 

#### export detected text regions

In [17]:
exported_file_paths = craft.export_detected_regions(
    image_path=image_path,
    image=image,
    regions=prediction_result["boxes"],
    output_dir=output_dir,
    rectify=True
)

#### export heatmap, detection points, box visualization

In [30]:
# gives (image)_text_detection.txt file
craft.export_extra_results(
    image_path=image_path,
    image=image,
    regions=prediction_result["boxes"],
    heatmaps=prediction_result["heatmaps"],
    output_dir=output_dir
)